### Experiments using IMLY ###

This notebook contains experimental runs of IMLY with different datasets.  
The readings of these experiments can be referred to in this [sheet](https://docs.google.com/spreadsheets/d/1E5jcq2w42gN8bMIaeaRJpAdhgSVN-2XDJ_YTHe4qfwY/edit?usp=sharing).

---

### Dataset  #1

#### Diabetes dataset

1 - Data preparation

In [47]:
# Gather dataset #




In [1]:
import experiment_automation_script
from os import path
import pandas as pd

dataset_info = experiment_automation_script.get_url("diabetes")
url = "../data/diabetes.csv" if path.exists("../data/diabetes.csv") else dataset_info['url']
data = pd.read_csv(url, delimiter=",", header=None, index_col=False)

X = data.iloc[:,:-1]
Y = data.iloc[:,-1:]

experiment_automation_script.dopify(dataset_info, 'linear_regression', X, Y, 0.60)

Using TensorFlow backend.


AttributeError: 'SklearnKerasRegressor' object has no attribute '__call__'

In [3]:
from sklearn.model_selection import train_test_split
import copy

model_name = 'linear_regression'
model_mappings = {
    'linear_regression': 'LinearRegression',
    'logistic_regression': 'LogisticRegression'
}

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

for key, value in model_mappings.items():
    if key == model_name:
        name = value

module = __import__('sklearn.linear_model', fromlist=[name])
imported_module = getattr(module, name)
model = imported_module

primal_model = model()

# Primal
primal_model.fit(x_train, y_train)
primal_model.__class__.__name__

'LinearRegression'

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

diabetes = datasets.load_diabetes()

# Use only one feature
diabetes_X = diabetes.data
# sc = StandardScaler()
# diabetes.data = sc.fit_transform(diabetes.data)

X = diabetes.data
Y = diabetes.target

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

# # Split the data into training/testing sets
# x_train = diabetes_X[:-20]
# x_test = diabetes_X[-20:]

# # Split the targets into training/testing sets
# y_train = diabetes.target[:-20]
# y_test = diabetes.target[-20:]


In [23]:
# temp_data = np.column_stack([X,Y])
# np.savetxt("diabetes.csv", temp_data, delimiter=",")